In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class GELU(nn.Module):
    def __init__(self):
        super(GELU, self).__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(1.702 * x) * x


In [4]:
class Exponential(nn.Module):
    """Custom exponential activation"""
    def forward(self, x):
        return torch.exp(x)

In [5]:
def activate(activation):
    activation = activation.lower()
    if activation == 'relu':
        return nn.ReLU()
    elif activation == 'gelu':
        return nn.GELU()
    elif activation == 'sigmoid':
        return nn.Sigmoid()
    elif activation == 'tanh':
        return nn.Tanh()
    elif activation == 'exponential':  # Custom implementation
        return Exponential()
    elif activation == 'softplus':
        return nn.Softplus()
    else:
        raise ValueError(f'Unrecognized activation "{activation}"')

In [6]:
# Initialize your input tensor
input_tensor = torch.randn((1, 3, 224, 224))

# Get the activation layer
activation_layer = activate('gelu')  # replace 'gelu' with the activation you want

# Apply the activation to your input tensor
output = activation_layer(input_tensor)

# Conv layer

In [8]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels, num_filters, kernel_size, padding='same', activation='relu', dropout=0.2):
        super(ConvLayer, self).__init__()

        # Calculate padding for 'same'
        if padding == 'same':
            padding = kernel_size // 2

        # Convolutional layer
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=num_filters, kernel_size=kernel_size,
                              stride=1, padding=padding, bias=False)

        # Batch normalization
        self.bn = nn.BatchNorm1d(num_features=num_filters)

        # Activation
        self.activation = activate(activation)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.activation(x)
        x = self.dropout(x)
        return x


In [9]:
# Initialize your input tensor
input_tensor = torch.randn((10, 4, 2048))

# Create an instance of the ConvLayer
conv_layer = ConvLayer(4, 256, 15, padding='same', activation='relu', dropout=0.2)

# Apply the conv_layer to your input tensor
output = conv_layer(input_tensor)

# Conv block

In [20]:
class ConvBlock(nn.Module):
    def __init__(
        self, 
        in_channels, 
        filters=None, 
        kernel_size=1, 
        activation='relu', 
        activation_end=None,
        strides=1, 
        dilation_rate=1, 
        l2_scale=0, 
        dropout=0, 
        conv_type='standard', 
        residual=False,
        pool_size=1, 
        batch_norm=False, 
        bn_momentum=0.99, 
        bn_gamma=None, 
        padding='same', 
        w1=False
    ):
        super(ConvBlock, self).__init__()
        self.residual = residual
        self.pool_size = pool_size
        self.padding = padding if padding != 'same' else kernel_size // 2
        self.batch_norm = batch_norm

        if filters is None:
            filters = in_channels

        # Choose convolution type
        if conv_type == 'separable':
            # PyTorch doesn't have a direct equivalent of Keras's SeparableConv1D.
            raise NotImplementedError("SeparableConv1D is not directly implemented in PyTorch.")
        elif w1:
            self.conv = nn.Conv2d(in_channels, filters, kernel_size, stride=strides, padding=self.padding, dilation=dilation_rate, bias=False)
        else:
            self.conv = nn.Conv1d(in_channels, filters, kernel_size, stride=strides, padding=self.padding, dilation=dilation_rate, bias=False)

        # Batch normalization
        if batch_norm:
            self.bn = nn.BatchNorm1d(num_features=filters, momentum=bn_momentum)  # Adjust for 2D if using Conv2d

        # Activation
        self.activation = activate(activation)  # Use the activate function from previous example
        self.activation_end = activate(activation_end) if activation_end else None

        # Dropout
        if dropout > 0:
            self.dropout = nn.Dropout(dropout)
        else:
            self.dropout = None

        # Pooling
        if pool_size > 1:
            if w1:
                self.pool = nn.MaxPool2d(kernel_size=pool_size, padding=self.padding)
            else:
                self.pool = nn.MaxPool1d(kernel_size=pool_size, padding=self.padding)

    def forward(self, x):
        residual = x if self.residual else None

        x = self.conv(x)

        if self.batch_norm:
            x = self.bn(x)

        x = self.activation(x)

        if self.dropout:
            x = self.dropout(x)

        if self.residual:
            x = x + residual

        if self.activation_end:
            x = self.activation_end(x)

        if self.pool_size > 1:
            x = self.pool(x)

        return x


In [21]:
# Initialize your input tensor
input_tensor = torch.randn((10, 4, 2048))

# Create an instance of the ConvBlock
conv_block = ConvBlock(4, filters=256, kernel_size=15, padding='same', activation='relu', dropout=0.2)

# Apply the conv_block to your input tensor
output = conv_block(input_tensor)


# Conv tower

In [22]:
import math

class ConvTower(nn.Module):
    def __init__(self, in_channels, filters_init, filters_mult=1, repeat=1, **kwargs):
        super(ConvTower, self).__init__()
        self.layers = nn.ModuleList()
        rep_filters = filters_init

        for ri in range(repeat):
            # Round filters to the nearest integer for the current block
            current_filters = int(round(rep_filters))
            # Create a new convolution block with the specified number of filters
            conv_block_layer = ConvBlock(in_channels=in_channels if ri == 0 else current_filters,
                                         filters=current_filters,
                                         **kwargs)
            self.layers.append(conv_block_layer)
            # Update in_channels for the next block and rep_filters for the entire sequence
            in_channels = current_filters
            rep_filters *= filters_mult

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [23]:
# Initialize your input tensor
input_tensor = torch.randn((10, 4, 2048))

# Create an instance of the ConvTower
conv_tower = ConvTower(in_channels=4, filters_init=64, filters_mult=1, repeat=3)

# Apply the conv_tower to your input tensor
output = conv_tower(input_tensor)

In [25]:
conv_tower

ConvTower(
  (layers): ModuleList(
    (0): ConvBlock(
      (conv): Conv1d(4, 64, kernel_size=(1,), stride=(1,), bias=False)
      (activation): ReLU()
    )
    (1-2): 2 x ConvBlock(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (activation): ReLU()
    )
  )
)

In [24]:
output.shape

torch.Size([10, 64, 2048])